## Final models on simulated datasets (Training on zone 1 and testing on zone 8)
### Energy consumption normalized and GHI added 

##### *Same experiments as in file RF_OneClass_Final.ipynb to compare the results*

In [211]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, accuracy_score

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter

In [214]:
train_x = X_train_zone1_simulated
train_y = y_train_zone1_simulated

test_x = X_test_zone8_simulated
test_y = y_test_zone8_simulated

# Define zones and variables for plots 
test_temp = 'Mean air temperature, degC, zone 8'
test_heat = 'Local heating units, W, zone 8'

## 3)  Fault 1 implemented 1 time during a year 

In [227]:
df_simulated = pd.read_csv('/Users/sararhiger/Desktop/Master thesis /Data/Processed data/Performance VS faults/1_fault1_dataprocessed.csv')


In [228]:
# Define the start and end times for training data
start_time_train = "2023-01-01 00:00:00"  
end_time_train = "2023-12-31 23:00:00"  

# Define the start and end times for testing data
start_time_test = "2023-01-01 00:00:00"  
end_time_test = "2023-12-31 23:00:00"

# Filter DataFrame for timestamps within the range
df_train_simulated = df_simulated[(df_simulated['Time'] >= start_time_train) & (df_simulated['Time'] <= end_time_train)]

df_test_simulated = df_simulated[(df_simulated['Time'] >= start_time_test) & (df_simulated['Time'] <= end_time_test)]

# Zone 1
X_train_zone1_simulated = df_train_simulated[['Global Horizontal Irradiance, W/m2','Outdoor air temp','Mean air temperature, degC, zone 1', 'Relative humidity, %, zone 1', 'CO2, ppm (vol), zone 1', 'Local heating units, W, zone 1']]
y_train_zone1_simulated = df_train_simulated['FDD, zone 1']
X_test_zone1_simulated = df_test_simulated[['Global Horizontal Irradiance, W/m2','Outdoor air temp','Mean air temperature, degC, zone 1', 'Relative humidity, %, zone 1', 'CO2, ppm (vol), zone 1', 'Local heating units, W, zone 1']]
y_test_zone1_simulated = df_test_simulated['FDD, zone 1']

# Zone 8
X_train_zone8_simulated = df_train_simulated[['Global Horizontal Irradiance, W/m2', 'Outdoor air temp','Mean air temperature, degC, zone 8', 'Relative humidity, %, zone 8', 'CO2, ppm (vol), zone 8', 'Local heating units, W, zone 8']]
y_train_zone8_simulated = df_train_simulated['FDD, zone 8']
X_test_zone8_simulated = df_test_simulated[['Global Horizontal Irradiance, W/m2', 'Outdoor air temp','Mean air temperature, degC, zone 8', 'Relative humidity, %, zone 8', 'CO2, ppm (vol), zone 8', 'Local heating units, W, zone 8']]
y_test_zone8_simulated = df_test_simulated['FDD, zone 8']

### Random Forest 

In [230]:
# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)#, class_weight='balanced')

# Train the model
rf_model.fit(train_x, train_y)

# Make predictions
y_pred_train = rf_model.predict(train_x)
y_pred = rf_model.predict(test_x)

# Evaluate the model
print("Accuracy:", accuracy_score(train_y, y_pred_train))
print("Train Report:\n", classification_report(train_y, y_pred_train))
print()
print("Accuracy:", accuracy_score(test_y, y_pred))
print("\nClassification Report:\n", classification_report(test_y, y_pred))

print(rf_model.feature_importances_)

Accuracy: 1.0
Train Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      8756
           1       1.00      1.00      1.00         4

    accuracy                           1.00      8760
   macro avg       1.00      1.00      1.00      8760
weighted avg       1.00      1.00      1.00      8760


Accuracy: 0.9985159817351598

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      8747
           1       0.00      0.00      0.00        13

    accuracy                           1.00      8760
   macro avg       0.50      0.50      0.50      8760
weighted avg       1.00      1.00      1.00      8760

[0.04808016 0.03983222 0.40635051 0.05549586 0.04922903 0.40101221]


/Users/sararhiger/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [231]:
# Store all runs
results = []

for i in range(10):
    rf_model = RandomForestClassifier()
    rf_model.fit(train_x, train_y)

    y_pred = rf_model.predict(test_x)
    report = classification_report(test_y, y_pred, output_dict=True)

    # Store per-class metrics
    run_result = {'run': i + 1}
    for label in report:
        if label not in ('accuracy', 'macro avg', 'weighted avg'):
            run_result[f'precision_class_{label}'] = report[label]['precision']
            run_result[f'recall_class_{label}'] = report[label]['recall']
            run_result[f'f1_class_{label}'] = report[label]['f1-score']
    
    results.append(run_result)

# Results as DataFrame
results_df = pd.DataFrame(results)

# Summary (mean and std)
summary = results_df.describe().loc[['mean', 'std']]

print("\nSummary (Mean ± Std):")
print(summary)

print("\nAll Per-Class Results:")
print(results_df)

/Users/sararhiger/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/sararhiger/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/sararhiger/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/sararhiger/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined


Summary (Mean ± Std):
          run  precision_class_0  recall_class_0  f1_class_0  \
mean  5.50000           0.998562             1.0    0.999280   
std   3.02765           0.000096             0.0    0.000048   

      precision_class_1  recall_class_1  f1_class_1  
mean           0.200000        0.030769    0.053333  
std            0.421637        0.064867    0.112437  

All Per-Class Results:
   run  precision_class_0  recall_class_0  f1_class_0  precision_class_1  \
0    1           0.998516             1.0    0.999257                0.0   
1    2           0.998744             1.0    0.999372                1.0   
2    3           0.998516             1.0    0.999257                0.0   
3    4           0.998744             1.0    0.999372                1.0   
4    5           0.998516             1.0    0.999257                0.0   
5    6           0.998516             1.0    0.999257                0.0   
6    7           0.998516             1.0    0.999257             

/Users/sararhiger/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [232]:
fig = go.Figure()

# Base temperature curve
fig.add_trace(go.Scatter(
    x=df_simulated['Time'], 
    y=df_simulated[test_temp], 
    mode='lines', 
    name='Temperature (C°)',
    yaxis='y1'
    ))

# Convert y_pred to a Pandas Series with the correct index
y_pred_series = pd.Series(y_pred, index=test_y.index)

# Find indices for different cases
correct_1_indices = (test_y == 1) & (y_pred_series == 1)  # True 1, Pred 1 -> Green
false_negative_indices = (test_y == 1) & (y_pred_series == 0)  # True 1, Pred 0 -> Red
false_positive_indices = (test_y == 0) & (y_pred_series == 1)  # True 0, Pred 1 -> Orange

# Extract timestamps
correct_1_timestamps = df_simulated.loc[test_y.index, 'Time'][correct_1_indices]
false_negative_timestamps = df_simulated.loc[test_y.index, 'Time'][false_negative_indices]
false_positive_timestamps = df_simulated.loc[test_y.index, 'Time'][false_positive_indices]

# Extract corresponding temperatures
correct_1_temps = df_simulated.loc[test_y.index, test_temp][correct_1_indices]
false_negative_temps = df_simulated.loc[test_y.index, test_temp][false_negative_indices]
false_positive_temps = df_simulated.loc[test_y.index, test_temp][false_positive_indices]

# Add markers for correct predictions (Green)
fig.add_trace(go.Scatter(
    x=correct_1_timestamps,
    y=correct_1_temps,
    mode='markers',
    marker=dict(color='green', size=10),
    name='Correct (4 predicted as 4)'
))

# Add markers for false negatives (Red)
fig.add_trace(go.Scatter(
    x=false_negative_timestamps,
    y=false_negative_temps,
    mode='markers',
    marker=dict(color='red', size=10, symbol='x'),
    name='False Negative (4 predicted as 0)'
))

# Add markers for false positives (Orange)
fig.add_trace(go.Scatter(
    x=false_positive_timestamps,
    y=false_positive_temps,
    mode='markers',
    marker=dict(color='orange', size=10, symbol='triangle-up'),
    name='False Positive (0 predicted as 4)'
))

# Customize layout
#fig.update_layout(
#    title="Temperature with low setpoint fault simulated in Zone 8 with prediction markers",
#    xaxis_title="Time",
 #   yaxis_title="Temperature (°C)",
   # legend_title="Legend",
    #template="plotly"
#)

# Update layout with secondary y-axis
fig.update_layout(
    title="Temperature & heating consumption in zone 8 with lowered setpoint simulated",
    xaxis_title="Time",
    yaxis=dict(
        title="Temperature (°C)",
        titlefont=dict(color="black"),
        tickfont=dict(color="black")
    ),
    yaxis2=dict(
        title="Heating Consumption (kWh)",  # Adjust unit if needed
        overlaying='y',
        side='right',
        titlefont=dict(color="black"),
        tickfont=dict(color="black"),
    ),
    legend=dict(
        x=1.05,         # Pushes legend to the right (1.0 is the edge of the plot area)
        y=1,            # Top alignment
        xanchor="left", # Anchor relative to left of the legend box
        yanchor="top"
    ),
    legend_title="Legend",
    template="plotly",
    height=600,
    width=1200,
)

# Show the plot

## 4) Fault 1 implemented 3 times during a year 

In [233]:
df_simulated = pd.read_csv('/Users/sararhiger/Desktop/Master thesis /Data/Processed data/Performance VS faults/3_Fault1_dataprocessed.csv')

In [234]:
# Define the start and end times for training data
start_time_train = "2023-01-01 00:00:00"  
end_time_train = "2023-12-31 23:00:00"  

# Define the start and end times for testing data
start_time_test = "2023-01-01 00:00:00"  
end_time_test = "2023-12-31 23:00:00"

# Filter DataFrame for timestamps within the range
df_train_simulated = df_simulated[(df_simulated['Time'] >= start_time_train) & (df_simulated['Time'] <= end_time_train)]

df_test_simulated = df_simulated[(df_simulated['Time'] >= start_time_test) & (df_simulated['Time'] <= end_time_test)]

# Zone 1
X_train_zone1_simulated = df_train_simulated[['Global Horizontal Irradiance, W/m2','Outdoor air temp','Mean air temperature, degC, zone 1', 'Relative humidity, %, zone 1', 'CO2, ppm (vol), zone 1', 'Local heating units, W, zone 1']]
y_train_zone1_simulated = df_train_simulated['FDD, zone 1']
X_test_zone1_simulated = df_test_simulated[['Global Horizontal Irradiance, W/m2','Outdoor air temp','Mean air temperature, degC, zone 1', 'Relative humidity, %, zone 1', 'CO2, ppm (vol), zone 1', 'Local heating units, W, zone 1']]
y_test_zone1_simulated = df_test_simulated['FDD, zone 1']

# Zone 8
X_train_zone8_simulated = df_train_simulated[['Global Horizontal Irradiance, W/m2', 'Outdoor air temp','Mean air temperature, degC, zone 8', 'Relative humidity, %, zone 8', 'CO2, ppm (vol), zone 8', 'Local heating units, W, zone 8']]
y_train_zone8_simulated = df_train_simulated['FDD, zone 8']
X_test_zone8_simulated = df_test_simulated[['Global Horizontal Irradiance, W/m2', 'Outdoor air temp','Mean air temperature, degC, zone 8', 'Relative humidity, %, zone 8', 'CO2, ppm (vol), zone 8', 'Local heating units, W, zone 8']]
y_test_zone8_simulated = df_test_simulated['FDD, zone 8']

In [235]:
train_x = X_train_zone1_simulated
train_y = y_train_zone1_simulated

test_x = X_test_zone8_simulated
test_y = y_test_zone8_simulated

# Define zones and variables for plots 
test_temp = 'Mean air temperature, degC, zone 8'
test_heat = 'Local heating units, W, zone 8'

### Random forest

In [236]:
# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)#, class_weight='balanced')

# Train the model
rf_model.fit(train_x, train_y)

# Make predictions
y_pred_train = rf_model.predict(train_x)
y_pred = rf_model.predict(test_x)

# Evaluate the model
print("Accuracy:", accuracy_score(train_y, y_pred_train))
print("Train Report:\n", classification_report(train_y, y_pred_train))
print()
print("Accuracy:", accuracy_score(test_y, y_pred))
print("\nClassification Report:\n", classification_report(test_y, y_pred))

print(rf_model.feature_importances_)

Accuracy: 1.0
Train Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      8748
           1       1.00      1.00      1.00        12

    accuracy                           1.00      8760
   macro avg       1.00      1.00      1.00      8760
weighted avg       1.00      1.00      1.00      8760


Accuracy: 0.9987442922374429

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      8747
           1       1.00      0.15      0.27        13

    accuracy                           1.00      8760
   macro avg       1.00      0.58      0.63      8760
weighted avg       1.00      1.00      1.00      8760

[0.05309099 0.05362027 0.44811837 0.0305583  0.02282299 0.39178908]


In [237]:
# Store all runs
results = []

for i in range(10):
    rf_model = RandomForestClassifier()
    rf_model.fit(train_x, train_y)

    y_pred = rf_model.predict(test_x)
    report = classification_report(test_y, y_pred, output_dict=True)

    # Store per-class metrics
    run_result = {'run': i + 1}
    for label in report:
        if label not in ('accuracy', 'macro avg', 'weighted avg'):
            run_result[f'precision_class_{label}'] = report[label]['precision']
            run_result[f'recall_class_{label}'] = report[label]['recall']
            run_result[f'f1_class_{label}'] = report[label]['f1-score']
    
    results.append(run_result)

# Results as DataFrame
results_df = pd.DataFrame(results)

# Summary (mean and std)
summary = results_df.describe().loc[['mean', 'std']]

print("\nSummary (Mean ± Std):")
print(summary)

print("\nAll Per-Class Results:")
print(results_df)


Summary (Mean ± Std):
          run  precision_class_0  recall_class_0  f1_class_0  \
mean  5.50000           0.998869             1.0    0.999434   
std   3.02765           0.000100             0.0    0.000050   

      precision_class_1  recall_class_1  f1_class_1  
mean                1.0        0.238462    0.380735  
std                 0.0        0.067353    0.089087  

All Per-Class Results:
   run  precision_class_0  recall_class_0  f1_class_0  precision_class_1  \
0    1           0.998858             1.0    0.999429                1.0   
1    2           0.998972             1.0    0.999486                1.0   
2    3           0.998972             1.0    0.999486                1.0   
3    4           0.998858             1.0    0.999429                1.0   
4    5           0.998972             1.0    0.999486                1.0   
5    6           0.998972             1.0    0.999486                1.0   
6    7           0.998744             1.0    0.999372             

In [238]:
# Compute confusion matrix
cm = confusion_matrix(test_y, y_pred)

# Ensure labels are 0 and 4
labels = [0, 1]  

# Create the confusion matrix heatmap
fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=labels,  # Predicted labels
    y=labels,  # Actual labels
    colorscale='Blues',
    showscale=True,
    text=cm,  # Display numbers in the cells
    texttemplate="%{text}",  # Ensure counts appear inside the cells
    hovertemplate='Actual: %{y}<br>Predicted: %{x}<br>Count: %{z}<extra></extra>'
))

# Update layout for binary classification (0 and 1 on axes)
fig.update_layout(
    title="Confusion Matrix for zone 8 fault 4 ",
    width=500,   # Reduce width
    height=400,
    xaxis=dict(
        title="Predicted Label",
        tickmode='array',        # Custom tick labels
        tickvals=[0, 1],         # Only show 0 and 1
        showgrid=False
    ),
    yaxis=dict(
        title="Actual Label",
        tickmode='array',
        tickvals=[0, 1],         # Only show 0 and 1
        showgrid=False
    ),
    template="plotly_white"
)

# Show the plot
fig.show()

In [239]:
fig = go.Figure()

# Base temperature curve
fig.add_trace(go.Scatter(
    x=df_simulated['Time'], 
    y=df_simulated[test_heat], 
    mode='lines', 
    name='Zone 8 faulty operation'
))

# Convert y_pred to a Pandas Series with the correct index
y_pred_series = pd.Series(y_pred, index=test_y.index)

# Find indices for different cases
correct_1_indices = (test_y == 1) & (y_pred_series == 1)  # True 1, Pred 1 -> Green
false_negative_indices = (test_y == 1) & (y_pred_series == 0)  # True 1, Pred 0 -> Red
false_positive_indices = (test_y == 0) & (y_pred_series == 1)  # True 0, Pred 1 -> Orange

# Extract timestamps
correct_1_timestamps = df_simulated.loc[test_y.index, 'Time'][correct_1_indices]
false_negative_timestamps = df_simulated.loc[test_y.index, 'Time'][false_negative_indices]
false_positive_timestamps = df_simulated.loc[test_y.index, 'Time'][false_positive_indices]

# Extract corresponding temperatures
correct_1_temps = df_simulated.loc[test_y.index, test_heat][correct_1_indices]
false_negative_temps = df_simulated.loc[test_y.index, test_heat][false_negative_indices]
false_positive_temps = df_simulated.loc[test_y.index, test_heat][false_positive_indices]

# Add markers for correct predictions (Green)
fig.add_trace(go.Scatter(
    x=correct_1_timestamps,
    y=correct_1_temps,
    mode='markers',
    marker=dict(color='green', size=10),
    name='Correct (1 predicted as 1)'
))

# Add markers for false negatives (Red)
fig.add_trace(go.Scatter(
    x=false_negative_timestamps,
    y=false_negative_temps,
    mode='markers',
    marker=dict(color='red', size=10, symbol='x'),
    name='False Negative (1 predicted as 0)'
))

# Add markers for false positives (Orange)
fig.add_trace(go.Scatter(
    x=false_positive_timestamps,
    y=false_positive_temps,
    mode='markers',
    marker=dict(color='orange', size=10, symbol='triangle-up'),
    name='False Positive (0 predicted as 1)'
))

# Customize layout
fig.update_layout(
    title="Heating consumption with faulty operation in zone 8 & prediction markers",
    xaxis_title="Time",
    yaxis_title="Heat (kWh)",
    legend_title="Legend",
    template="plotly", 
    height = 500,
    width = 1200
)

# Show the plot

## 5) Fault 1 implemented 7 times during a year

In [240]:
df_simulated = pd.read_csv('/Users/sararhiger/Desktop/Master thesis /Data/Processed data/Performance VS faults/7_Fault1_dataprocessed.csv')

In [241]:
# Define the start and end times for training data
start_time_train = "2023-01-01 00:00:00"  
end_time_train = "2023-12-31 23:00:00"  

# Define the start and end times for testing data
start_time_test = "2023-01-01 00:00:00"  
end_time_test = "2023-12-31 23:00:00"

# Filter DataFrame for timestamps within the range
df_train_simulated = df_simulated[(df_simulated['Time'] >= start_time_train) & (df_simulated['Time'] <= end_time_train)]

df_test_simulated = df_simulated[(df_simulated['Time'] >= start_time_test) & (df_simulated['Time'] <= end_time_test)]

# Zone 1
X_train_zone1_simulated = df_train_simulated[['Global Horizontal Irradiance, W/m2','Outdoor air temp','Mean air temperature, degC, zone 1', 'Relative humidity, %, zone 1', 'CO2, ppm (vol), zone 1', 'Local heating units, W, zone 1']]
y_train_zone1_simulated = df_train_simulated['FDD, zone 1']
X_test_zone1_simulated = df_test_simulated[['Global Horizontal Irradiance, W/m2','Outdoor air temp','Mean air temperature, degC, zone 1', 'Relative humidity, %, zone 1', 'CO2, ppm (vol), zone 1', 'Local heating units, W, zone 1']]
y_test_zone1_simulated = df_test_simulated['FDD, zone 1']

# Zone 8
X_train_zone8_simulated = df_train_simulated[['Global Horizontal Irradiance, W/m2', 'Outdoor air temp','Mean air temperature, degC, zone 8', 'Relative humidity, %, zone 8', 'CO2, ppm (vol), zone 8', 'Local heating units, W, zone 8']]
y_train_zone8_simulated = df_train_simulated['FDD, zone 8']
X_test_zone8_simulated = df_test_simulated[['Global Horizontal Irradiance, W/m2', 'Outdoor air temp','Mean air temperature, degC, zone 8', 'Relative humidity, %, zone 8', 'CO2, ppm (vol), zone 8', 'Local heating units, W, zone 8']]
y_test_zone8_simulated = df_test_simulated['FDD, zone 8']

In [242]:
train_x = X_train_zone1_simulated
train_y = y_train_zone1_simulated

test_x = X_test_zone8_simulated
test_y = y_test_zone8_simulated

# Define zones and variables for plots 
test_temp = 'Mean air temperature, degC, zone 8'
test_heat = 'Local heating units, W, zone 8'

#### Random forest 

In [243]:
# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)#, class_weight='balanced')

# Train the model
rf_model.fit(train_x, train_y)

# Make predictions
y_pred_train = rf_model.predict(train_x)
y_pred = rf_model.predict(test_x)

# Evaluate the model
print("Accuracy:", accuracy_score(train_y, y_pred_train))
print("Train Report:\n", classification_report(train_y, y_pred_train))
print()
print("Accuracy:", accuracy_score(test_y, y_pred))
print("\nClassification Report:\n", classification_report(test_y, y_pred))

print(rf_model.feature_importances_)

Accuracy: 1.0
Train Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      8735
           1       1.00      1.00      1.00        25

    accuracy                           1.00      8760
   macro avg       1.00      1.00      1.00      8760
weighted avg       1.00      1.00      1.00      8760


Accuracy: 0.998972602739726

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      8747
           1       0.83      0.38      0.53        13

    accuracy                           1.00      8760
   macro avg       0.92      0.69      0.76      8760
weighted avg       1.00      1.00      1.00      8760

[0.04583885 0.0329603  0.46175509 0.02932602 0.028271   0.40184875]


In [244]:
# Store all runs
results = []

for i in range(10):
    rf_model = RandomForestClassifier()
    rf_model.fit(train_x, train_y)

    y_pred = rf_model.predict(test_x)
    report = classification_report(test_y, y_pred, output_dict=True)

    # Store per-class metrics
    run_result = {'run': i + 1}
    for label in report:
        if label not in ('accuracy', 'macro avg', 'weighted avg'):
            run_result[f'precision_class_{label}'] = report[label]['precision']
            run_result[f'recall_class_{label}'] = report[label]['recall']
            run_result[f'f1_class_{label}'] = report[label]['f1-score']
    
    results.append(run_result)

# Results as DataFrame
results_df = pd.DataFrame(results)

# Summary (mean and std)
summary = results_df.describe().loc[['mean', 'std']]

print("\nSummary (Mean ± Std):")
print(summary)

print("\nAll Per-Class Results:")
print(results_df)


Summary (Mean ± Std):
          run  precision_class_0  recall_class_0  f1_class_0  \
mean  5.50000           0.999086    9.998857e-01    0.999486   
std   3.02765           0.000000    2.340556e-16    0.000000   

      precision_class_1  recall_class_1  f1_class_1  
mean       8.333333e-01        0.384615    0.526316  
std        1.170278e-16        0.000000    0.000000  

All Per-Class Results:
   run  precision_class_0  recall_class_0  f1_class_0  precision_class_1  \
0    1           0.999086        0.999886    0.999486           0.833333   
1    2           0.999086        0.999886    0.999486           0.833333   
2    3           0.999086        0.999886    0.999486           0.833333   
3    4           0.999086        0.999886    0.999486           0.833333   
4    5           0.999086        0.999886    0.999486           0.833333   
5    6           0.999086        0.999886    0.999486           0.833333   
6    7           0.999086        0.999886    0.999486           0.

In [245]:
fig = go.Figure()

# Base temperature curve
fig.add_trace(go.Scatter(
    x=df_simulated['Time'], 
    y=df_simulated[test_heat], 
    mode='lines', 
    name='Zone 8 faulty operation'
))

# Convert y_pred to a Pandas Series with the correct index
y_pred_series = pd.Series(y_pred, index=test_y.index)

# Find indices for different cases
correct_1_indices = (test_y == 1) & (y_pred_series == 1)  # True 1, Pred 1 -> Green
false_negative_indices = (test_y == 1) & (y_pred_series == 0)  # True 1, Pred 0 -> Red
false_positive_indices = (test_y == 0) & (y_pred_series == 1)  # True 0, Pred 1 -> Orange

# Extract timestamps
correct_1_timestamps = df_simulated.loc[test_y.index, 'Time'][correct_1_indices]
false_negative_timestamps = df_simulated.loc[test_y.index, 'Time'][false_negative_indices]
false_positive_timestamps = df_simulated.loc[test_y.index, 'Time'][false_positive_indices]

# Extract corresponding temperatures
correct_1_temps = df_simulated.loc[test_y.index, test_heat][correct_1_indices]
false_negative_temps = df_simulated.loc[test_y.index, test_heat][false_negative_indices]
false_positive_temps = df_simulated.loc[test_y.index, test_heat][false_positive_indices]

# Add markers for correct predictions (Green)
fig.add_trace(go.Scatter(
    x=correct_1_timestamps,
    y=correct_1_temps,
    mode='markers',
    marker=dict(color='green', size=10),
    name='Correct (1 predicted as 1)'
))

# Add markers for false negatives (Red)
fig.add_trace(go.Scatter(
    x=false_negative_timestamps,
    y=false_negative_temps,
    mode='markers',
    marker=dict(color='red', size=10, symbol='x'),
    name='False Negative (1 predicted as 0)'
))

# Add markers for false positives (Orange)
fig.add_trace(go.Scatter(
    x=false_positive_timestamps,
    y=false_positive_temps,
    mode='markers',
    marker=dict(color='orange', size=10, symbol='triangle-up'),
    name='False Positive (0 predicted as 1)'
))

# Customize layout
fig.update_layout(
    title="Heating consumption with faulty operation in zone 8 & prediction markers",
    xaxis_title="Time",
    yaxis_title="Heat (kWh)",
    legend_title="Legend",
    template="plotly", 
    height = 500,
    width = 1200
)

# Show the plot